In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import datetime
import seaborn as sns
import pickle
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
%matplotlib inline
from sklearn.utils import resample
import re
import nltk
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
train = pd.read_csv("gdrive/My Drive/Colab Notebooks/Kaggle/edsa-holiday-challenge/train.csv")
test = pd.read_csv("gdrive/My Drive/Colab Notebooks/Kaggle/edsa-holiday-challenge/test.csv")
train.head(10)

,ID,ItemCategory,ItemCategory_New
0,0,Inj 1.50 x Metacam 5mg/ml Injection for Dogs a...,anti-inflammatory injection
1,1,Mini metabolic profile,blood test
2,2,Biochemistry profile A,blood test
3,3,Comfortan Injection (CD Sch2) 4.35pm dose,analgesic injections
4,4,Micralax Enema 5ml,non specific medications
5,5,Chloramphenicol 0.5% eye drops *fridge*,topical eye medications
6,6,Feline-Skin Stitch Up-Simple,surgical time (cost per hour)
7,7,LAB BACT. CULTURE AND SENS SWAB (IDEXX),laboratory fees
8,8,idexx haematology comp,blood test
9,9,INJECTION CAP,fluids


In [0]:
# train['ItemCategory_New'] = train['ItemCategory_New'].apply(lambda x: x.lower())
# train['ItemCategory_New'] =train['ItemCategory_New'].apply(lambda x: x.replace('unable to group as no description','other'))
# train.head(10)

In [0]:
# train.ItemCategory_New.value_counts()

In [6]:
test.head()

,ID,ItemCategory
0,84485,Consultation - Repeat (HPC 10% Sales Discount)
1,84486,1 x Lab Lasercyte Haematology
2,84487,5x Metronidazole 200mg Tablets
3,84488,Subsequent Extra Large Radiograph
4,84489,30X SYNOQUIN MEDIUM TASTY TABLETS


# Text Preprocessing

In [0]:
y = train[['ItemCategory_New']]
test_ID = test['ID']
# Now drop the  'Id' colums before the feature engineering process
train.drop(['ItemCategory_New','ID'], axis=1, inplace=True)
test.drop(['ID'], axis=1, inplace=True)

In [8]:
y.head()

,ItemCategory_New
0,anti-inflammatory injection
1,blood test
2,blood test
3,analgesic injections
4,non specific medications


In [9]:
train.head()

,ItemCategory
0,Inj 1.50 x Metacam 5mg/ml Injection for Dogs a...
1,Mini metabolic profile
2,Biochemistry profile A
3,Comfortan Injection (CD Sch2) 4.35pm dose
4,Micralax Enema 5ml


In [10]:
test.head()

,ItemCategory
0,Consultation - Repeat (HPC 10% Sales Discount)
1,1 x Lab Lasercyte Haematology
2,5x Metronidazole 200mg Tablets
3,Subsequent Extra Large Radiograph
4,30X SYNOQUIN MEDIUM TASTY TABLETS


In [11]:
# Put all the features to gether for transformation
features = pd.concat([train, test],sort=False).reset_index(drop=True)
features.head()

,ItemCategory
0,Inj 1.50 x Metacam 5mg/ml Injection for Dogs a...
1,Mini metabolic profile
2,Biochemistry profile A
3,Comfortan Injection (CD Sch2) 4.35pm dose
4,Micralax Enema 5ml


In [12]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Text to Features

**Create Features**

In [0]:
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

lem = WordNetLemmatizer()
# ps = PorterStemmer()
corpus =[]
# cleaning text
for i in range(0,len(features['ItemCategory'])):

    item = re.sub('[^a-zA-Z]', ' ',str(features.iloc[i].values)) 
    item = item.lower()
    item = item.split()
    stop_wrds = set(stopwords.words('english'))
    item = [lem.lemmatize(word, "v") for word in item if not word in stop_wrds]
    item = ' '.join(item)
    corpus.append(item)

features['item_clean'] = corpus
features.drop(['ItemCategory'], axis=1, inplace=True)         

In [14]:
features.head()

,item_clean
0,inj x metacam mg ml injection dog cat ml bottl...
1,mini metabolic profile
2,biochemistry profile
3,comfortan injection cd sch pm dose
4,micralax enema ml


In [15]:
# Statistical Features
obj = TfidfVectorizer(ngram_range =(2,2), analyzer = 'char_wb')
final_features = obj.fit_transform(features['item_clean'])
print(final_features[0])

# obj = CountVectorizer()
# final_features = obj.fit_transform(features['item_clean'])
# print(final_features[0])

  (0, 240)	0.10418635020330713
  (0, 622)	0.15611505765167244
  (0, 414)	0.13898480979922512
  (0, 326)	0.10474974084459919
  (0, 143)	0.13038408639067917
  (0, 131)	0.060535032439626886
  (0, 316)	0.0893822950704924
  (0, 524)	0.18204587995066407
  (0, 532)	0.17238656529665744
  (0, 410)	0.14935369901626502
  (0, 68)	0.15362418261434344
  (0, 1)	0.1040048093482683
  (0, 512)	0.07573722606406456
  (0, 46)	0.08314838501760824
  (0, 2)	0.07135445592188894
  (0, 397)	0.10634910060675908
  (0, 120)	0.10929320073269456
  (0, 3)	0.08708274791497325
  (0, 363)	0.07327418738211959
  (0, 404)	0.0796147207505411
  (0, 252)	0.08546671467973334
  (0, 521)	0.07894977895023825
  (0, 98)	0.11235973047019691
  (0, 134)	0.11197935971911321
  (0, 269)	0.13339733208797735
  (0, 311)	0.2053467768715015
  (0, 350)	0.2847523572183857
  (0, 185)	0.1467365893637852
  (0, 345)	0.09075511397512427
  (0, 338)	0.2080402286040721
  (0, 39)	0.22316180377488715
  (0, 79)	0.2722967337760842
  (0, 29)	0.11823053870761

In [16]:
# create our target variables
y=y.apply(lambda x: x.astype(str).str.lower())
y.head()

,ItemCategory_New
0,anti-inflammatory injection
1,blood test
2,blood test
3,analgesic injections
4,non specific medications


**Create The Target Variable**

In [17]:
y.ItemCategory_New.value_counts()

ultrasound                               7783
physiotherapy                            7382
blood test                               6755
consultation fee                         5379
laboratory fees                          4234
fluids                                   3548
anaesthetic (general) induction          3363
anti-biotic tablets                      3015
analgesic injections                     2397
hospitalisation (cost per hour)          2308
surgical time (cost per hour)            2244
anti-biotic injection                    2215
other                                    2090
sedation                                 2059
x-rays                                   2056
anti-inflammatory injection              1950
anti-inflammatory tablets                1889
gastro-intestinal treatments             1348
heart condition medications              1245
skin/allergy medications                 1242
analgesic tablets                        1167
suture material                   

In [18]:
y_unique = y.ItemCategory_New.value_counts()
y_unique = pd.DataFrame(y_unique)
y_unique.head()

,ItemCategory_New
ultrasound,7783
physiotherapy,7382
blood test,6755
consultation fee,5379
laboratory fees,4234


In [19]:
y_unique = y_unique.reset_index(drop=False)
y_unique.head()

,index,ItemCategory_New
0,ultrasound,7783
1,physiotherapy,7382
2,blood test,6755
3,consultation fee,5379
4,laboratory fees,4234


In [20]:
y_unique.drop(['ItemCategory_New'], axis=1, inplace=True)  
y_unique['New_ID'] = y_unique.index
y_unique.head()

,index,New_ID
0,ultrasound,0
1,physiotherapy,1
2,blood test,2
3,consultation fee,3
4,laboratory fees,4


In [21]:
y_unique.tail()

,index,New_ID
66,emetic drugs,66
67,pre-medications (if invoiced as),67
68,unable to group as no description,68
69,blood transfusion,69
70,referral fee,70


In [22]:
len(y_unique)

71

In [23]:
y.head()

,ItemCategory_New
0,anti-inflammatory injection
1,blood test
2,blood test
3,analgesic injections
4,non specific medications


In [24]:
mapping = dict(y_unique[['index', 'New_ID']].values)
y['Item_ID'] = y.ItemCategory_New.map(mapping)
y.head()

,ItemCategory_New,Item_ID
0,anti-inflammatory injection,15
1,blood test,2
2,blood test,2
3,analgesic injections,8
4,non specific medications,22


In [25]:
y.drop(['ItemCategory_New'], axis=1, inplace=True)
y.head()

,Item_ID
0,15
1,2
2,2
3,8
4,22


In [26]:
len(y)

84485

In [27]:
# Spliting the data back to train(X,y) and test(X_sub)
X = final_features[:len(y), :]
X_test_final = final_features[len(y):, :]
print('Features size for train(X,y) and test(X_test):')
print('X', X.shape, 'y', y.shape, 'X_test', X_test_final.shape)

Features size for train(X,y) and test(X_test):
X (84485, 685) y (84485, 1) X_test (14909, 685)


# Text Classification

In [0]:
from sklearn.multiclass import OneVsOneClassifier

In [0]:
 X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.33)

In [0]:
from sklearn.svm import LinearSVC
model = LinearSVC(multi_class='ovr',dual=False, fit_intercept=False)

In [31]:
model.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [32]:
# y_pred = sc_y.inverse_transform(model.predict(X_test))
y_pred = model.predict(X_test)
y_pred

array([11, 15, 13, ..., 34,  4,  3])

In [33]:
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Mean squared error: 107.37


In [34]:
# performance matrics and model eveluation using sklearn.metrics inbuilt classification metric
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.57      0.60      2527
           1       0.60      0.59      0.59      2399
           2       0.80      0.89      0.84      2233
           3       0.78      0.79      0.78      1812
           4       0.80      0.83      0.82      1438
           5       0.85      0.92      0.88      1167
           6       0.83      0.87      0.85      1149
           7       0.88      0.93      0.90      1006
           8       0.90      0.95      0.93       801
           9       0.88      0.94      0.91       747
          10       0.68      0.56      0.61       750
          11       0.86      0.93      0.89       754
          12       0.76      0.66      0.71       732
          13       0.79      0.83      0.81       687
          14       0.90      0.86      0.88       708
          15       0.87      0.95      0.91       635
          16       0.85      0.86      0.86       653
          17       0.76    

**Final model**

In [35]:
model.fit(X, y.values.ravel())

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [36]:
# c = sc_y.inverse_transform(model.predict(X_test_final))
y_predicted = model.predict(X_test_final)
y_predicted = pd.DataFrame({'PredictItem_ID':y_predicted})
y_predicted.head()

,PredictItem_ID
0,3
1,2
2,7
3,1
4,32


In [37]:
mapping = dict(y_unique[['New_ID','index']].values)
y_predicted['ItemCategory_New'] = y_predicted.PredictItem_ID.map(mapping)
y_predicted.head()

,PredictItem_ID,ItemCategory_New
0,3,consultation fee
1,2,blood test
2,7,anti-biotic tablets
3,1,physiotherapy
4,32,supplements


In [38]:
y_predicted = np.array(y_predicted.drop('PredictItem_ID', axis=1))
y_predicted

array([['consultation fee'],
       ['blood test'],
       ['anti-biotic tablets'],
       ...,
       ['consultation fee'],
       ['x-rays'],
       ['consultation fee']], dtype=object)

# Submit

In [0]:
submission = pd.DataFrame({'ID' : np.array(test_ID),'ItemCategory_New' : y_predicted.ravel()})

In [40]:
submission.head()

,ID,ItemCategory_New
0,84485,consultation fee
1,84486,blood test
2,84487,anti-biotic tablets
3,84488,physiotherapy
4,84489,supplements


In [0]:
submission.to_csv("gdrive/My Drive/Colab Notebooks/Kaggle/edsa-holiday-challenge/new_submission.csv", index=False)